In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import random
import nltk
from nltk.chat.util import Chat, reflections
import warnings
warnings.filterwarnings('ignore')

# Ensure NLTK data is downloaded
nltk.download('punkt')

# Custom reflections for more empathetic responses
counselor_reflections = {
    "i am": "you are",
    "i was": "you were",
    "i": "you",
    "i'm": "you are",
    "i'd": "you would",
    "i've": "you have",
    "i'll": "you will",
    "my": "your",
    "you are": "I am",
    "you were": "I was",
    "you've": "I have",
    "you'll": "I will",
    "your": "my",
    "yours": "mine",
    "you": "me",
    "me": "you"
}

# Pairs of patterns and responses for the counselor
counseling_pairs = [
    [
        r"(.*)depressed(.*)|(.*)sad(.*)|(.*)unhappy(.*)|(.*)down(.*)",
        ["I'm sorry to hear you're feeling this way. Would you like to talk more about what's making you feel %2?",
         "It sounds like you're going through a difficult time. Can you tell me more about these feelings?",
         "Depression can be really challenging. Have you considered reaching out to a professional about this?",
         "Feeling down is tough. What kind of support do you feel you need right now?",
         "It's okay to not be okay. What's been on your mind lately?"]
    ],
    [
        r"(.*)anxious(.*)|(.*)nervous(.*)|(.*)stressed(.*)|(.*)worried(.*)",
        ["Anxiety can feel overwhelming. What specifically is making you feel %2 right now?",
         "It's understandable to feel anxious sometimes. What coping strategies have you tried?",
         "Would it help to talk through what's causing your anxiety?",
         "Stress can really weigh on you. What are some things that usually help you relax?",
         "Feeling worried is a common experience. What's the biggest concern on your mind?"]
    ],
    [
        r"(.*)lonely(.*)|(.*)alone(.*)|(.*)isolated(.*)",
        ["Feeling lonely can be really painful. Have you been able to connect with others recently?",
         "You're not alone in feeling this way. Would you like to talk about what's making you feel isolated?",
         "Loneliness is a common human experience, but that doesn't make it easier. What would help you feel more connected?",
         "It sounds like you're feeling disconnected. What do you miss about connecting with others?",
         "Being alone can be hard. What are some things you enjoy doing that might help you feel less isolated?"]
    ],
    [
        r"(.*)angry(.*)|(.*)mad(.*)|(.*)frustrated(.*)",
        ["Anger is a natural emotion. Can you tell me more about what's causing these feelings?",
         "It sounds like you're feeling really frustrated. What would help you feel better in this situation?",
         "Anger often comes from unmet needs. What do you think might be behind your feelings of anger?",
         "It's okay to feel angry. What happened that made you feel this way?",
         "Frustration can be tough to manage. What do you usually do when you feel this way?"]
    ],
    [
        r"(.*)suicidal(.*)|(.*)kill myself(.*)|(.*)end it all(.*)|(.*)want to die(.*)|(.*)end my life(.*)",
        ["I'm really concerned about what you're saying. Your life is valuable. Please contact a suicide hotline immediately.",
         "This sounds very serious. I strongly encourage you to reach out to emergency services or a trusted person right now.",
         "I care about your safety. Please call a crisis hotline or tell someone you trust about these feelings immediately."]
    ],
    [
        r"(.*)happy(.*)|(.*)joyful(.*)|(.*)excited(.*)|(.*)good(.*)",
        ["That's wonderful to hear! What's making you feel %2 today?",
         "I'm so glad you're feeling good! Tell me more about it.",
         "It's great to hear some positivity! What's been going well for you?",
         "Sharing happy feelings is lovely. What's the best part of your day?",
         "That's fantastic! What do you think contributed to you feeling so %2?"]
    ],
     [
        r"(.*)better(.*)|(.*)improving(.*)|(.*)coping(.*)",
        ["That's great progress! What do you think is helping you feel %1?",
         "It sounds like things are getting better. What have you been doing differently?",
         "I'm happy to hear you're coping well. What strategies are working for you?",
         "That's a positive step! What do you want to build on from this?",
         "It takes strength to work towards feeling better. What are you most proud of in this journey?"]
    ],
    [
        r"(.*)help(.*)|(.*)support(.*)",
        ["I'm here to help. What specifically would you like support with?",
         "You're not alone in this. What kind of help you are looking for right now?",
         "I want to support you. Can you tell me more about what you need help with?",
         "What kind of support would be most helpful to you right now?",
         "I'm here to listen and offer support. What's on your mind?"]
    ],
    [
        r"(.*)thank(.*)|(.*)appreciate(.*)",
        ["You're very welcome. I'm glad I could be here for you.",
         "It's my pleasure to support you. How are you feeling now?",
         "Thank you for sharing with me. It takes courage to talk about these things.",
         "I appreciate you saying that. It means a lot.",
         "You're welcome! How can I continue to support you?"]
    ],
    [
        r"(.*)",
        ["I hear you. Can you tell me more about that?",
         "That sounds important. How does that make you feel?",
         "I'm listening. What else comes to mind when you think about this?",
         "Tell me more about what's on your mind.",
         "I'm here to listen without judgment. What would you like to share?"]
    ]
]

# Create the chatbot
counselor = Chat(counseling_pairs, counselor_reflections)

# Emergency resources including UAE and India contacts
emergency_resources = [
    "🆘 Global/International:",
    "International Association for Suicide Prevention: https://www.iasp.info/resources/Crisis_Centres/",
    "Befrienders Worldwide: https://www.befrienders.org",
    "",
    "🇺🇸 United States:",
    "National Suicide Prevention Lifeline: 988",
    "Crisis Text Line: Text HOME to 741741",
    "",
    "🇦🇪 United Arab Emirates:",
    "Dubai Health Authority Helpline: 800 342 (DHA)",
    "National Program for Happiness & Wellbeing - Mental Support: 800 4673 (HOPE)",
    "Al Amal Psychiatric Hospital: +971 4 706 4000",
    "",
    "🇮🇳 India:",
    "Vandrevala Foundation Helpline: 1860 2662 345 or 1800 2333 330",
    "AASRA: +91 98204 66766 (24/7)",
    "ICall: +91 91529 87821 (Mon-Sat, 10am-8pm)",
    "Sneha Foundation: +91 44 2464 0050 (24/7)",
    "",
    "🌐 Other Resources:",
    "Mental Health America: https://www.mhanational.org/get-help",
    "Therapy Aid: https://therapyaid.com"
]

class MentalHealthChatbot:
    def __init__(self):
        # Create widgets
        self.output = widgets.Output()
        self.input_text = widgets.Text(placeholder='Type your message here...', layout=widgets.Layout(width='80%'))
        self.send_button = widgets.Button(description='Send', button_style='primary')
        self.clear_button = widgets.Button(description='Clear', button_style='warning')
        self.resources_button = widgets.Button(description='Emergency Resources', button_style='danger')

        # Set up button click handlers
        self.send_button.on_click(self.send_message)
        self.clear_button.on_click(self.clear_chat)
        self.resources_button.on_click(self.show_resources)

        # Arrange widgets
        self.input_box = widgets.HBox([self.input_text, self.send_button])
        self.button_box = widgets.HBox([self.clear_button, self.resources_button])
        self.container = widgets.VBox([self.output, self.input_box, self.button_box])

        # Initial greeting
        with self.output:
            print("Counselor: Hello, I'm here to listen and support you. How are you feeling today?")

    def send_message(self, b):
        user_input = self.input_text.value
        self.input_text.value = ''  # Clear input field

        with self.output:
            print(f"You: {user_input}")

            # Check for emergency keywords
            emergency_keywords = ['suicide', 'kill myself', 'end my life', 'want to die', 'harm myself']
            if any(keyword in user_input.lower() for keyword in emergency_keywords):
                print("\nCounselor: I'm very concerned about what you're saying. Please know that help is available immediately.")
                print("Here are some emergency resources you can contact right now:")
                for resource in emergency_resources:
                    if resource:  # Skip empty strings used for spacing
                        print(f"- {resource}")
                print("\nYou are not alone, and people care about you.")

            # Get counselor response
            response = counselor.respond(user_input)
            print(f"Counselor: {response}")

    def clear_chat(self, b):
        self.output.clear_output()
        with self.output:
            print("Counselor: Hello again. I'm here to listen. How can I support you today?")

    def show_resources(self, b):
        with self.output:
            print("\nCounselor: Here are some mental health resources that might be helpful:")
            for resource in emergency_resources:
                if resource:  # Skip empty strings used for spacing
                    print(f"- {resource}")
            print("\nRemember, it's okay to ask for help when you need it.")

    def display(self):
        display(self.container)

# Create and display the chatbot
chatbot = MentalHealthChatbot()
chatbot.display()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
